In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
# Set filepaths

path_to_data = '../../data/sampling/icpms/'
path_to_angle = '../../data/angles/'

cores = ['alhic2302','alhic2302','alhic2302','alhic2302','alhic1901']
sections = ['155_2','156_2','158','159_3','230_4']

#cores = ['alhic1901']
#sections = ['230_4']


In [8]:
# Load metadata
meta = pd.read_excel(path_to_data + 'icpms_metadata.xlsx')
meta['section'] = meta['section'].astype(str)

# load science data
df = pd.read_csv(path_to_data + 'icpms_raw.csv')
print(df.columns)
df['top_depth'] = df['Top Depth (cm)'] / 100
df['bot_depth'] = df['Bot Depth (cm)'] / 100
df['ave_depth'] = (df['top_depth'] + df['bot_depth']) / 2
df['top_depth_adj'] = df['top_depth']
df['bot_depth_adj'] = df['bot_depth']
df['ave_depth_adj'] = df['ave_depth']
df['y_m'] = 0
df['x_m'] = 0

df = df.drop(columns=['Top Depth (cm)','Bot Depth (cm)'])

# load angle data
core = 'alhic2302'
angles = pd.read_csv(path_to_angle+core+'_deepangles_means.csv',index_col=0)

Index(['ID', 'core', 'section', 'cut', 'Top Depth (cm)', 'Bot Depth (cm)',
       'Li', 'Rb', 'Sr', 'Zr', 'Mo', 'Cd', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
       'Sm', 'Eu', 'Tb', 'Gd', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Pb', 'Th',
       'U', 'Be', 'W', 'Na', 'Mg', 'Al', 'P', 'Ca', 'Ti', 'V', 'Cr', 'Mn',
       'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'S', 'Si', 'K', 'As'],
      dtype='object')


In [9]:
# Apply dip adnjustment

# loop through sticks and core/section combos
for core,section in zip(cores,sections):

    # get the angle of the core, apply a dip adjustment if angle data exists
    if section in angles.index or section == '230_4':

        print("Processing section "+section)

        run_angle = True

        if section == '230_4':
            x_angle = -30
            y_angle = -30
        else:
            x_angle = angles.at[section,'AC-tr-mean-angle']
            y_angle = angles.at[section,'AC-r-mean-angle']

        # get cut data
        meta_row = meta[(meta['core']==core) & (meta['section']==section)]
        y = (meta_row['y_hi'].to_numpy() + meta_row['y_lo'].to_numpy())/2
        x = (meta_row['x_hi'].to_numpy() + meta_row['x_lo'].to_numpy())/2
        top_depth = meta_row['top_depth'].to_numpy() + meta_row['offset_mm'].to_numpy()/1000

        print("Top Depth "+str(top_depth))

        #print(meta_row)

        # compute shift due to layer angle adjustment
        y_shift = -y * np.tan(y_angle*np.pi/180)
        x_shift = x * np.tan(x_angle*np.pi/180)
        shift = y_shift + x_shift
        print("     y_shift: "+str(y_shift))
        print("     x_shift: "+str(x_shift))        
        print("     Shift for section "+ section + " is "+str(shift))

    else:
        print("Angle not found for section "+ section)
        run_angle = False



    if run_angle:
        df.loc[(df['core'] == core) & (df['section'] == section) , 'top_depth_adj'] += top_depth[0]+shift/1000
        df.loc[(df['core'] == core) & (df['section'] == section), 'bot_depth_adj'] += top_depth[0]+shift/1000
        df.loc[(df['core'] == core) & (df['section'] == section), 'ave_depth_adj'] += top_depth[0]+shift/1000
        df.loc[(df['core'] == core) & (df['section'] == section), 'top_depth'] += top_depth[0]
        df.loc[(df['core'] == core) & (df['section'] == section), 'bot_depth'] += top_depth[0]
        df.loc[(df['core'] == core) & (df['section'] == section), 'ave_depth'] += top_depth[0]
        df.loc[(df['core'] == core) & (df['section'] == section), 'y_m'] += y/1000
        df.loc[(df['core'] == core) & (df['section'] == section), 'x_m'] += x/1000
    else:
        print("ERROR")

Processing section 155_2
Top Depth [126.389]
     y_shift: [-12.35561698]
     x_shift: [25.52295261]
     Shift for section 155_2 is [13.16733563]
Processing section 156_2
Top Depth [127.688]
     y_shift: [-1.26539215]
     x_shift: [56.29553126]
     Shift for section 156_2 is [55.03013911]
Processing section 158
Top Depth [128.715]
     y_shift: [-1.24271306]
     x_shift: [26.06860739]
     Shift for section 158 is [24.82589433]
Processing section 159_3
Top Depth [129.82]
     y_shift: [6.67907644]
     x_shift: [-31.76073337]
     Shift for section 159_3 is [-25.08165693]
Processing section 230_4
Top Depth [156.799]
     y_shift: [9.23760431]
     x_shift: [-26.55811238]
     Shift for section 230_4 is [-17.32050808]


In [10]:
# wrap up - reordering columns, saving to csv

# reorder columns
columns_to_move = ['core','section','ave_depth','ave_depth_adj','y_m','x_m']
columns = df.columns.tolist()
for col in columns_to_move:
    columns.remove(col)
columns = columns_to_move + columns
df = df[columns]

# save to csv
df.to_csv(path_to_data + 'master_icpms.csv',index=False)